In [1]:
#!pip install geojson
#!conda install -c conda-forge folium=0.5.0 --yes 
#!conda install -c conda-forge geopy --yes 
#!conda install yellowbrick --yes
#!conda install geopandas --yes


In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

import geopandas as gp

import json # library to handle JSON files

import geojson as gs #libary to handle GeoJSON

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

# import k-means from clustering stage
from sklearn.cluster import KMeans

#from yellowbrick.cluster import KElbowVisualizer

import folium # map rendering library

#import plotly.graph_objects as go

from sklearn.metrics import silhouette_samples, silhouette_score

print('Libraries imported.')

Libraries imported.


In [3]:
CLIENT_ID = 'L0OFOAI4LU1E23YHMY5BKT2IEF1Y51OURMKKJ5GDSBCHQR5Y' # your Foursquare ID
CLIENT_SECRET = '2UDLBVN3E3YTIHOOF004XGQVEG35XFT2AJJCK0QDMKFHX0JY' # your Foursquare Secret
VERSION = '20190605' # Foursquare API version
LIMIT = 100 #setting limit

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: L0OFOAI4LU1E23YHMY5BKT2IEF1Y51OURMKKJ5GDSBCHQR5Y
CLIENT_SECRET:2UDLBVN3E3YTIHOOF004XGQVEG35XFT2AJJCK0QDMKFHX0JY


In [4]:
address = 'Norwich, UK'

geolocator = Nominatim(user_agent="norwich")
location = geolocator.geocode(address)
lat_1 = location.latitude
long_1 = location.longitude
print('{} co-ordinates: {}, {}.'.format(address, lat_1, long_1))

Norwich, UK co-ordinates: 52.628606, 1.29227.


In [5]:
address = 'Cambridge, UK'

geolocator = Nominatim(user_agent="cambridge")
location = geolocator.geocode(address)
lat_2 = location.latitude
long_2 = location.longitude
print('{} co-ordinates: {}, {}.'.format(address, lat_2, long_2))

Cambridge, UK co-ordinates: 52.2034823, 0.1235817.


In [6]:
map_norwich = folium.Map(location=[lat_1, long_1], zoom_start=15)
map_cambridge = folium.Map(location=[lat_2, long_2], zoom_start=15)

In [7]:
map_cambridge

In [9]:
url_cam="https://opendata.arcgis.com/datasets/da831f80764346889837c72508f046fa_0.geojson?where=UPPER(lsoa11nm)%20like%20'%25CAMBRIDGE%25'"
cam=gp.read_file(url_cam)


DriverError: '/vsimem/6eee818fb0914d439583308bb2f84705' not recognized as a supported file format.

In [ ]:
cam

In [ ]:
shire=cam[cam['lsoa11nm'].str.contains("Cambridgeshire")].index
shire

In [ ]:
cam=cam.drop(shire)
cam

In [ ]:
url_nor="https://opendata.arcgis.com/datasets/da831f80764346889837c72508f046fa_0.geojson?where=UPPER(lsoa11nm)%20like%20'%25NORWICH%25'"
nor=gp.read_file(url_nor)

In [ ]:
nor

In [ ]:
def getXY(pt):
    return (pt.x, pt.y)
centroidseries = nor['geometry'].centroid
x,y = [list(t) for t in zip(*map(getXY, centroidseries))]

In [ ]:
df=pd.DataFrame(nor)

In [ ]:
df

In [ ]:
df["long"]=x
df["lat"]=y
df

In [ ]:
df1=df.drop(columns=["geometry"])
df2=df1.rename(columns={"lsoa11nm":"Neighborhood"})
df2

In [ ]:
for lat, lng in zip(df2['lat'], df2['long']):
    #label = '{}'.format(area)
    #label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        #popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_norwich)  

In [ ]:
map_norwich

In [ ]:
def getXY(pt):
    return (pt.x, pt.y)

centroidseries = cam.centroid

x,y = [list(t) for t in zip(*map(getXY, centroidseries))]

df=pd.DataFrame(cam)
df

In [ ]:
df["long"]=x
df["lat"]=y
df

In [ ]:
df3=df.drop(columns=["geometry"])
df3

In [ ]:
for lat, lng in zip(df3['lat'], df3['long']):
    #label = '{}'.format(area)
    #label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        #popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_cambridge)  

In [ ]:
map_cambridge

In [ ]:
radius=500


In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, long in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            long, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
norwich_venues = getNearbyVenues(names=df2['Neighborhood'],
                                   latitudes=df2['lat'],
                                   longitudes=df2['long']
                                  )


In [ ]:
norwich_venues.head()

In [ ]:
norwich_venues.groupby('Venue Category').count()

In [ ]:
# one hot encoding
norwich_onehot = pd.get_dummies(norwich_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
norwich_onehot['Neighborhood'] = norwich_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [norwich_onehot.columns[-1]] + list(norwich_onehot.columns[:-1])
norwich_onehot = norwich_onehot[fixed_columns]

norwich_onehot.head()

In [ ]:
norwich_groupedd = norwich_onehot.groupby('Neighborhood').mean().reset_index()
norwich_groupedd

In [ ]:
norwich_grouped=norwich_groupedd.drop([0]).reset_index()
norwich_grouped


In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = norwich_grouped['Neighborhood']

for ind in np.arange(norwich_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(norwich_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

In [ ]:
# set number of clusters
kclusters = 5

norwich_grouped_clustering = norwich_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(norwich_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_[0:10])
print(kmeans.score(norwich_grouped_clustering))


In [ ]:
norwich_grouped_clustering

In [ ]:
Ks = range(1, 10)
km = [KMeans(n_clusters=i) for i in Ks]
score = [km[i].fit(norwich_grouped_clustering).score(norwich_grouped_clustering) for i in range(len(km))]
plt.plot(Ks, score)

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


In [ ]:
for i, k in enumerate([2,5,10]):
    fig, ax1 = plt.subplots(1)
    fig.set_size_inches(18, 7)
    
    # Run the Kmeans algorithm
    km = KMeans(n_clusters=k)
    labels = km.fit_predict(norwich_grouped_clustering)
    centroids = km.cluster_centers_

    # Get silhouette samples
    silhouette_vals = silhouette_samples(norwich_grouped_clustering, labels)

    # Silhouette plot
    y_ticks = []
    y_lower, y_upper = 0, 0
    for i, cluster in enumerate(np.unique(labels)):
        cluster_silhouette_vals = silhouette_vals[labels == cluster]
        cluster_silhouette_vals.sort()
        y_upper += len(cluster_silhouette_vals)
        ax1.barh(range(y_lower, y_upper), cluster_silhouette_vals, edgecolor='none', height=1)
        ax1.text(-0.03, (y_lower + y_upper) / 2, str(i + 1))
        y_lower += len(cluster_silhouette_vals)

    # Get the average silhouette score and plot it
    avg_score = np.mean(silhouette_vals)
    ax1.axvline(avg_score, linestyle='--', linewidth=2, color='green')
    ax1.set_yticks([])
    ax1.set_xlim([-0.1, 1])
    ax1.set_xlabel('Silhouette coefficient values')
    ax1.set_ylabel('Cluster labels')
    ax1.set_title('Silhouette plot for the various clusters', y=1.02);
    


In [ ]:
cambridge_venues = getNearbyVenues(names=df3['Neighborhood'],
                                   latitudes=df3['lat'],
                                   longitudes=df3['long']
                                  )

cambridge_venues

In [ ]:
# one hot encoding
cambridge_onehott = pd.get_dummies(cambridge_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
cambridge_onehott['Neighborhood'] = cambridge_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [cambridge_onehott.columns[-1]] + list(cambridge_onehott.columns[:-1])
cambridge_onehott = cambridge_onehott[fixed_columns]

cambridge_onehott

In [ ]:
cambridge_onehott.loc[cambridge_onehott['Airport Lounge'] == 1]

In [ ]:
shire=cambridge_onehott[cambridge_onehott['Neighborhood'].str.contains("Cambridgeshire")].index

In [ ]:
cambridge_onehot=cambridge_onehott.drop(shire)
cambridge_onehot

In [ ]:
#Group oneshot by mean
cambridge_grouped = cambridge_onehot.groupby('Neighborhood').mean().reset_index()

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
cambridge_venues_sorted = pd.DataFrame(columns=columns)
cambridge_venues_sorted['Neighborhood'] =cambridge_grouped['Neighborhood']

for ind in np.arange(cambridge_grouped.shape[0]):
    cambridge_venues_sorted.iloc[ind, 1:] = return_most_common_venues(cambridge_grouped.iloc[ind, :], num_top_venues)

cambridge_venues_sorted.head()


In [ ]:
Ks = range(1, 10)
km = [KMeans(n_clusters=i) for i in Ks]
score = [km[i].fit(cambridge_grouped_clustering).score(cambridge_grouped_clustering) for i in range(len(km))]
plt.plot(Ks, score)

In [ ]:
for i, k in enumerate([2,5,7]):
    fig, ax1 = plt.subplots(1)
    fig.set_size_inches(18, 7)
    
    # Run the Kmeans algorithm
    km = KMeans(n_clusters=k)
    labels = km.fit_predict(cambridge_grouped_clustering)
    centroids = km.cluster_centers_

    # Get silhouette samples
    silhouette_vals = silhouette_samples(cambridge_grouped_clustering, labels)

    # Silhouette plot
    y_ticks = []
    y_lower, y_upper = 0, 0
    for i, cluster in enumerate(np.unique(labels)):
        cluster_silhouette_vals = silhouette_vals[labels == cluster]
        cluster_silhouette_vals.sort()
        y_upper += len(cluster_silhouette_vals)
        ax1.barh(range(y_lower, y_upper), cluster_silhouette_vals, edgecolor='none', height=1)
        ax1.text(-0.03, (y_lower + y_upper) / 2, str(i + 1))
        y_lower += len(cluster_silhouette_vals)

    # Get the average silhouette score and plot it
    avg_score = np.mean(silhouette_vals)
    ax1.axvline(avg_score, linestyle='--', linewidth=2, color='green')
    ax1.set_yticks([])
    ax1.set_xlim([-0.1, 1])
    ax1.set_xlabel('Silhouette coefficient values')
    ax1.set_ylabel('Cluster labels')
    ax1.set_title('Silhouette plot for the various clusters', y=1.02);
    

In [ ]:
df2

In [ ]:
neighborhoods_venues_sorted.head()

In [ ]:
norwich_merged=neighborhoods_venues_sorted.merge(df2, on='Neighborhood')

In [ ]:
norwich_merged

In [ ]:
norwich_merged.astype({'Cluster Labels': 'int64'}).dtypes

In [ ]:
# create map
norwich_clusters = folium.Map(location=[lat_1, long_1], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(norwich_merged['lat'], norwich_merged['long'], norwich_merged['Neighborhood'], norwich_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(norwich_clusters)
    
folium.TileLayer('openstreetmap').add_to(norwich_clusters)
       
#norwich_clusters

In [ ]:
#add cluster labels

#rename 

df3=df3.rename(columns={"lsoa11nm": "Neighborhood"})

#merge venue and centroid data
cambridge_merged=cambridge_venues_sorted.merge(df3, on='Neighborhood')

# create map
cambridge_clusters = folium.Map(location=[lat_1, long_1], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(cambridge_merged['lat'], cambridge_merged['long'], cambridge_merged['Neighborhood'], cambridge_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(norwich_clusters)
    
folium.TileLayer('openstreetmap').add_to(cambridge_clusters)
       
cambridge_clusters

In [ ]:
df3

In [ ]:
norwich_merged.sort_values(by="Cluster Labels")

In [ ]:
norwich_merged.loc[norwich_merged['Cluster Labels'] == 0, norwich_merged.columns[[1] + list(range(2,7))]]